In [2]:
import pandas as pd


In [7]:
df_product

,Unnamed: 0,product_id,brand_id,family_id,"""group""",legacy_id,title
0,0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit
1,1,6e6b4255-006a-4d13-b312-3b1426290d8e,ce8d442c-e662-44f8-a130-2e436f119ede,6598b655-3e19-448c-8b4f-ff6a29447b34,P1393,9727,Poly Pant deeluxe
2,2,6549c1b1-989e-432a-bdc9-b603c438b310,cfca4b92-12a6-411c-a62a-a874d00aa735,1b93b30c-833f-45a8-ba69-c21d2e7f4824,J1738,16138,Onleden Sweater knt
3,3,dace6963-7e84-4cef-a153-3c6fb29cc7c3,1600fb81-6f8b-4fa2-9ae2-a33420125939,c6b81db8-29ed-43d6-b14d-0fd913d7ce36,K892,11621,Cerise Jacket print
4,4,d6679919-7a49-4edd-bd04-982289d0e8ac,09358432-a44c-4cb4-b8e6-bf51b403bf39,c59807ac-0362-4078-94b0-37ea3df6cdd0,S2517,14076,Auttum Shirt hippie
...,...,...,...,...,...,...,...
2927,2927,c70a703e-dc16-4ac4-8c14-fbba5660bcb1,921f8c67-2a93-4329-9247-c4153183b04b,1dc55e58-a5c3-462c-b73d-8dd480de90fc,P2199,15856,Gumm Pant place
2928,2928,9f3acaac-17b3-43a4-ae09-1677293d8ace,f620512a-1e55-4c69-8208-9c83ed4725a5,c59807ac-0362-4078-94b0-37ea3df6cdd0,S2385,15630,Kentucky Shirt melange
2929,2929,50701dda-a3f4-4b59-8698-da3757382574,1600fb81-6f8b-4fa2-9ae2-a33420125939,39792668-5cfd-42c2-8872-ad776e9bad70,T2299,17217,Lines Top cerise
2930,2930,22efc184-5ecb-401d-8dea-9569ceae492d,1600fb81-6f8b-4fa2-9ae2-a33420125939,9c5a3fee-dad0-444a-acb3-c059e2f3db20,P2253,17225,Jara Short pant


In [ ]:
import pandas as pd
import sys

# --- 1. FUNCIÓN DE CARGA INTELIGENTE (Comas y Puntos y Coma) ---
def cargar_flexible(ruta_archivo):
    print(f"-> Procesando {ruta_archivo}...")
    
    # Intento A: Detección automática del motor de Python (suele acertar)
    try:
        df = pd.read_csv(ruta_archivo, sep=None, engine='python')
    except:
        df = pd.DataFrame() # Si falla, inicializamos vacío
        
    # Intento B: Si falla o detecta solo 1 columna, forzamos punto y coma (;)
    if df.empty or df.shape[1] < 2:
        # print("   (Reintentando con separador ';')")
        try:
            df_semi = pd.read_csv(ruta_archivo, sep=';')
            if df_semi.shape[1] > 1:
                df = df_semi
        except:
            pass

    # Intento C: Si sigue siendo 1 columna, forzamos coma (,) ignorando comillas
    if df.empty or df.shape[1] < 2:
        # print("   (Reintentando con separador ',' ignorando comillas)")
        try:
            df_comma = pd.read_csv(ruta_archivo, sep=',', quoting=3)
            if df_comma.shape[1] > 1:
                df = df_comma
        except:
            pass
            
    # Limpieza de nombres de columnas (quita espacios y comillas)
    df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")
    return df

# --- 2. CARGA DE DATOS ---
# Ajusta las rutas si es necesario. Uso las que salieron en tu log.
base_path = "Datos/Datos looks/" 
# Si tus archivos están en la carpeta actual, borra "Datos/Datos looks/" y deja solo el nombre

try:
    df_product = cargar_flexible(f'{base_path}product_2.csv')
    df_variant = cargar_flexible(f'{base_path}product_variant.csv')
    df_color = cargar_flexible(f'{base_path}color.csv')
    df_size = cargar_flexible(f'{base_path}size.csv')
    df_brand = cargar_flexible(f'{base_path}brand.csv')
    df_prod_feat_val = cargar_flexible(f'{base_path}product_feature_value.csv')
    df_feat_val = cargar_flexible(f'{base_path}feature_value.csv')
    df_feat_val_fam = cargar_flexible(f'{base_path}feature_value_family.csv')
    df_feature = cargar_flexible(f'{base_path}feature.csv')
except FileNotFoundError:
    print("\n ERROR: No se encuentran los archivos en la ruta especificada.")
    print(" Por favor, edita la variable 'base_path' al inicio del script con tu ruta correcta.")
    sys.exit()

# --- 3. NORMALIZACIÓN DE CLAVES (CRÍTICO) ---
# Renombramos 'id' a 'product_id' explícitamente en la tabla product
if 'id' in df_product.columns: 
    df_product.rename(columns={'id': 'product_id'}, inplace=True)

if 'id' in df_variant.columns: 
    df_variant.rename(columns={'id': 'variant_id'}, inplace=True)

# Validación de seguridad antes del merge
print("\n--- ESTADO PREVIO AL MERGE ---")
print(f"Product: {df_product.shape} -> Tiene 'product_id': {'product_id' in df_product.columns}")
print(f"Variant: {df_variant.shape} -> Tiene 'product_id': {'product_id' in df_variant.columns}")

if 'product_id' not in df_product.columns:
    sys.exit(f"ERROR: df_product sigue sin tener 'product_id'. Columnas detectadas: {list(df_product.columns)}")

# --- 4. UNIFICACIÓN (MERGE) ---
print("\n--- UNIFICANDO DATAFRAMES ---")

# 4.1 Base
df_main = pd.merge(df_product, df_variant, on='product_id', how='left')

# 4.2 Color y Talla
df_color.rename(columns={'id': 'color_id_ref', 'title': 'color_name'}, inplace=True)
df_size.rename(columns={'id': 'size_id_ref', 'title': 'size_name'}, inplace=True)

# Asegurar tipos de datos para el cruce (str vs int)
for df in [df_main, df_color, df_size]:
    for col in ['color_id', 'color_id_ref', 'size_id', 'size_id_ref']:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace('.0', '', regex=False)

df_main = pd.merge(df_main, df_color, left_on='color_id', right_on='color_id_ref', how='left')
df_main = pd.merge(df_main, df_size, left_on='size_id', right_on='size_id_ref', how='left')

# 4.3 Marca
df_brand.rename(columns={'id': 'brand_id_ref', 'title': 'brand_name'}, inplace=True)
df_main = pd.merge(df_main, df_brand, left_on='brand_id', right_on='brand_id_ref', how='left')

# 4.4 Features (Características)
df_prod_feat_val.rename(columns={'id': 'pfv_id'}, inplace=True)
df_feat_val.rename(columns={'id': 'fv_id_ref', 'value': 'feature_value_name'}, inplace=True)
df_feature.rename(columns={'id': 'feature_id_ref', 'title': 'feature_name'}, inplace=True)
df_feat_val_fam.rename(columns={'id': 'fvf_id'}, inplace=True) 

# Cadena de Merges
df_final = pd.merge(df_main, df_prod_feat_val, on='product_id', how='left')
df_final = pd.merge(df_final, df_feat_val, left_on='feature_value_id', right_on='fv_id_ref', how='left')
df_final = pd.merge(df_final, df_feature, left_on='feature_id', right_on='feature_id_ref', how='left')
df_final = pd.merge(df_final, df_feat_val_fam, left_on='fv_id_ref', right_on='feature_value_id', how='left', suffixes=('', '_fam'))

# --- 5. LIMPIEZA Y RESULTADO ---
# Eliminar columnas técnicas basura
cols_drop = [c for c in df_final.columns if '_ref' in c or 'Unnamed' in c]
df_final.drop(columns=cols_drop, inplace=True, errors='ignore')

print(f"\n¡ÉXITO TOTAL! DataFrame unificado.")
print(f"Filas: {df_final.shape[0]}, Columnas: {df_final.shape[1]}")
display(df_final.head())

-> Procesando Datos/Datos looks/product_2.csv...
-> Procesando Datos/Datos looks/product_variant.csv...
-> Procesando Datos/Datos looks/color.csv...
-> Procesando Datos/Datos looks/size.csv...
-> Procesando Datos/Datos looks/brand.csv...
-> Procesando Datos/Datos looks/product_feature_value.csv...
-> Procesando Datos/Datos looks/feature_value.csv...
-> Procesando Datos/Datos looks/feature_value_family.csv...
-> Procesando Datos/Datos looks/feature.csv...

--- ESTADO PREVIO AL MERGE ---
Product: (2932, 7) -> Tiene 'product_id': True
Variant: (65226, 9) -> Tiene 'product_id': True

--- UNIFICANDO DATAFRAMES ---

¡ÉXITO TOTAL! DataFrame unificado.
Filas: 14286029, Columnas: 36


,product_id,brand_id,family_id,group,legacy_id_x,title,variant_id,barcode,color_id,legacy_id_y,...,pfv_id,feature_value_id,feature_value_name,feature_id,name,type,multiple,unit,family_id_fam,feature_value_id_fam
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,141f32d6-a5af-49bf-b019-fd28354977f1,b5ce1fed-4c38-4920-90b5-42e46f971e97
1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,1b93b30c-833f-45a8-ba69-c21d2e7f4824,b5ce1fed-4c38-4920-90b5-42e46f971e97
2,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,3087da56-5a27-4e58-aea7-1555c640f232,b5ce1fed-4c38-4920-90b5-42e46f971e97
3,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,39792668-5cfd-42c2-8872-ad776e9bad70,b5ce1fed-4c38-4920-90b5-42e46f971e97
4,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,95f835a3-03fe-40f1-a813-4fbfd2842845,b5ce1fed-4c38-4920-90b5-42e46f971e97


In [19]:
import pandas as pd
import sys
import os

# --- CONFIGURACIÓN ---
# Ajusta esto si tus archivos están en otra carpeta. 
# Si están junto al notebook, déjalo como "" o "."
RUTA_CARPETA = "Datos/Datos looks/"  

# --- 1. FUNCIÓN DE CARGA "TODOTERRENO" ---
# Esta función soluciona el problema de los separadores (; vs ,) automáticamente
def cargar_csv_inteligente(nombre_archivo):
    ruta_completa = os.path.join(RUTA_CARPETA, nombre_archivo)
    print(f"Leemos: {nombre_archivo} ...", end=" ")
    
    if not os.path.exists(ruta_completa):
        print(f"❌ ERROR: No encuentro el archivo en {ruta_completa}")
        return None

    try:
        # Intento 1: Dejar que Python detecte el separador
        df = pd.read_csv(ruta_completa, sep=None, engine='python')
        
        # Verificación: Si sale 1 sola columna, sospechamos mal formato y forzamos punto y coma
        if df.shape[1] < 2:
            df = pd.read_csv(ruta_completa, sep=';')
            
        # Limpieza: Quitamos espacios y comillas de los nombres de columnas
        df.columns = df.columns.str.strip().str.replace('"', '').str.replace("'", "")
        print(f"✅ OK ({df.shape[0]} filas, {df.shape[1]} cols)")
        return df
        
    except Exception as e:
        print(f"\n❌ Fallo crítico leyendo {nombre_archivo}: {e}")
        return None

# --- 2. CARGA DE DATOS ---
print("--- CARGANDO TABLAS ---")
df_product = cargar_csv_inteligente('product_2.csv')
df_variant = cargar_csv_inteligente('product_variant.csv')
df_color   = cargar_csv_inteligente('color.csv')
df_size    = cargar_csv_inteligente('size.csv')
df_brand   = cargar_csv_inteligente('brand.csv')
df_pfv     = cargar_csv_inteligente('product_feature_value.csv') # pfv = product_feature_value
df_fv      = cargar_csv_inteligente('feature_value.csv')         # fv = feature_value
df_fvf     = cargar_csv_inteligente('feature_value_family.csv')  # fvf = feature_value_family
df_feat    = cargar_csv_inteligente('feature.csv')               # feat = feature

# Verificación de seguridad: Si algo falló al cargar, paramos.
if any(df is None for df in [df_product, df_variant, df_color, df_size, df_brand]):
    sys.exit("⛔ DETENIDO: Faltan archivos obligatorios.")

# --- 3. PREPARACIÓN DE VARIABLES (RENOMBRADO) ---
# Renombramos las columnas 'id' originales para que coincidan con tu diagrama
# y evitar que Pandas se confunda al tener muchas columnas llamadas "id".

# Tabla PRODUCT (Amarillo) -> id es product_id
if 'id' in df_product.columns: df_product.rename(columns={'id': 'product_id'}, inplace=True)

# Tabla PRODUCT_VARIANT (Naranja/Verde) -> id es variant_id
if 'id' in df_variant.columns: df_variant.rename(columns={'id': 'variant_id'}, inplace=True)

# Tablas de referencia (Color, Talla, Marca)
if 'id' in df_color.columns: df_color.rename(columns={'id': 'color_id_ref', 'title': 'color_name'}, inplace=True)
if 'id' in df_size.columns:  df_size.rename(columns={'id': 'size_id_ref', 'title': 'size_name'}, inplace=True)
if 'id' in df_brand.columns: df_brand.rename(columns={'id': 'brand_id_ref', 'title': 'brand_name'}, inplace=True)

# Tablas de Features (Características)
if 'id' in df_pfv.columns:   df_pfv.rename(columns={'id': 'pfv_id'}, inplace=True)
if 'id' in df_fv.columns:    df_fv.rename(columns={'id': 'fv_id_ref', 'value': 'feature_value_name'}, inplace=True)
if 'id' in df_feat.columns:  df_feat.rename(columns={'id': 'feat_id_ref', 'title': 'feature_name'}, inplace=True)
# feature_value_family suele tener feature_value_id, no requiere renombre crítico de su ID propio si solo queremos cruzar.

# --- 4. MERGE (UNIFICACIÓN) ---
print("\n--- UNIFICANDO DATOS ---")

# A) BLOQUE CENTRAL: Producto + Variante
# Diagrama: product(id) -> product_variant(product_id)
df_main = pd.merge(df_product, df_variant, on='product_id', how='left')
print(f"1. Producto + Variante: {df_main.shape}")

# B) BLOQUE FÍSICO: + Color y Talla
# Aseguramos tipos compatibles (texto con texto) para evitar errores de cruce
df_main['color_id'] = df_main['color_id'].astype(str).str.replace('.0', '', regex=False)
df_color['color_id_ref'] = df_color['color_id_ref'].astype(str)
df_main['size_id'] = df_main['size_id'].astype(str).str.replace('.0', '', regex=False)
df_size['size_id_ref'] = df_size['size_id_ref'].astype(str)

# Diagrama: product_variant(color_id) -> color(id)
df_main = pd.merge(df_main, df_color, left_on='color_id', right_on='color_id_ref', how='left')
# Diagrama: product_variant(size_id) -> size(id)
df_main = pd.merge(df_main, df_size, left_on='size_id', right_on='size_id_ref', how='left')
print(f"2. + Color y Talla:     {df_main.shape}")

# C) BLOQUE MARCA: + Brand
# Diagrama: product(brand_id) -> brand(id)
df_main = pd.merge(df_main, df_brand, left_on='brand_id', right_on='brand_id_ref', how='left')
print(f"3. + Marca:             {df_main.shape}")

# D) BLOQUE CARACTERÍSTICAS (FEATURES) - Aquí el dataset crece a lo largo
# Diagrama: product(id) -> product_feature_value(product_id)
df_feats = pd.merge(df_main, df_pfv, on='product_id', how='left')

# Diagrama: product_feature_value(feature_value_id) -> feature_value(id)
df_feats = pd.merge(df_feats, df_fv, left_on='feature_value_id', right_on='fv_id_ref', how='left')

# Diagrama: feature_value(feature_id) -> feature(id)
df_feats = pd.merge(df_feats, df_feat, left_on='feature_id', right_on='feat_id_ref', how='left')

# Diagrama: feature_value(id) -> feature_value_family(feature_value_id)
# Nota: Según tu prompt "feature_value_family conecta con id=feature_value_id"
df_final = pd.merge(df_feats, df_fvf, left_on='fv_id_ref', right_on='feature_value_id', how='left', suffixes=('', '_fam'))
print(f"4. + Características:   {df_final.shape}")

# --- 5. LIMPIEZA FINAL ---
# Quitamos las columnas repetidas que se usaron solo para el cruce (las que terminan en _ref)
cols_borrar = [c for c in df_final.columns if '_ref' in c or 'Unnamed' in c]
df_final.drop(columns=cols_borrar, inplace=True, errors='ignore')

print("\n✅ ¡PROCESO TERMINADO!")
print(f"DataFrame final 'df_final': {df_final.shape[0]} filas y {df_final.shape[1]} columnas.")
display(df_final.head(3))

--- CARGANDO TABLAS ---
Leemos: product_2.csv ... ✅ OK (2932 filas, 7 cols)
Leemos: product_variant.csv ... ✅ OK (65226 filas, 9 cols)
Leemos: color.csv ... ✅ OK (54 filas, 5 cols)
Leemos: size.csv ... ✅ OK (126 filas, 6 cols)
Leemos: brand.csv ... ✅ OK (163 filas, 4 cols)
Leemos: product_feature_value.csv ... ✅ OK (75347 filas, 3 cols)
Leemos: feature_value.csv ... ✅ OK (2266 filas, 3 cols)
Leemos: feature_value_family.csv ... ✅ OK (34633 filas, 2 cols)
Leemos: feature.csv ... ✅ OK (61 filas, 5 cols)

--- UNIFICANDO DATOS ---
1. Producto + Variante: (23935, 15)
2. + Color y Talla:     (23935, 26)
3. + Marca:             (23935, 30)
4. + Características:   (14286029, 42)

✅ ¡PROCESO TERMINADO!
DataFrame final 'df_final': 14286029 filas y 36 columnas.


,product_id,brand_id,family_id,group,legacy_id_x,title,variant_id,barcode,color_id,legacy_id_y,...,pfv_id,feature_value_id,feature_value_name,feature_id,name,type,multiple,unit,family_id_fam,feature_value_id_fam
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,141f32d6-a5af-49bf-b019-fd28354977f1,b5ce1fed-4c38-4920-90b5-42e46f971e97
1,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,1b93b30c-833f-45a8-ba69-c21d2e7f4824,b5ce1fed-4c38-4920-90b5-42e46f971e97
2,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,cfca4b92-12a6-411c-a62a-a874d00aa735,141f32d6-a5af-49bf-b019-fd28354977f1,J1214,\N,Onljade Cardigan knit,fbc2f719-e038-4243-892c-7bd2ca6e80b5,740661,6183657f-3a82-449d-aa6a-a14e96b0e9dd,\N,...,17180364-12d1-423b-b9f4-ed7d121910d2,b5ce1fed-4c38-4920-90b5-42e46f971e97,cardigan,0ccb5374-db8c-4dbb-afae-6b2652af170d,top_type,STRING,f,\N,3087da56-5a27-4e58-aea7-1555c640f232,b5ce1fed-4c38-4920-90b5-42e46f971e97


In [35]:
dffinal = df_final[['product_id','title','season','name_x','lookiero','name_y','feature_value_name','name']]
dffinal = dffinal.drop_duplicates()
dffinal

,product_id,title,season,name_x,lookiero,name_y,feature_value_name,name
0,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,"{""size"": ""XL"", ""format"": ""XL""}",BRAND119,cardigan,top_type
6,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,"{""size"": ""XL"", ""format"": ""XL""}",BRAND119,ribbed,finishing
36,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,"{""size"": ""XL"", ""format"": ""XL""}",BRAND119,two,adventurous
66,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,"{""size"": ""XL"", ""format"": ""XL""}",BRAND119,without_closing,closing
96,5f5c4a50-557a-4d53-9e81-cc72fdd18fbf,Onljade Cardigan knit,8,black,"{""size"": ""XL"", ""format"": ""XL""}",BRAND119,cold_season,weather
...,...,...,...,...,...,...,...,...
14285906,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,"{""size"": ""M"", ""format"": ""M""}",BRAND54,skinny,down_part_type
14285909,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,"{""size"": ""M"", ""format"": ""M""}",BRAND54,cotton,composition
14285939,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,"{""size"": ""M"", ""format"": ""M""}",BRAND54,elastane,composition
14285969,98c51c8b-c467-4025-9517-806334bc8680,Sculp Jeans skinny,7,blue_dark,"{""size"": ""M"", ""format"": ""M""}",BRAND54,smooth,print
